![](../images/xor_and_or_nand.png)

## Minimum Integer Set Algorithm
- construct linear program that solves the constraints for group of logic gates

In [16]:
# find the size of the minimum set of integer numbers for representing AND, OR, NAND with a neuron and sigmoid activation function
import gurobipy as g
import itertools


NUMBER_OF_INTEGERS = 3
NUMBER_OF_HELPER_VARIABLES = pow(NUMBER_OF_INTEGERS, NUMBER_OF_INTEGERS)
# Big-M
M = 100

model = g.Model()

# define number of variables
int_set = model.addVars(NUMBER_OF_INTEGERS, vtype=g.GRB.INTEGER, lb=-15, ub=15, name="x")


# generate all possible permutations with repetition of integer variables
numbers = list(range(NUMBER_OF_INTEGERS))
all_permutations = list(itertools.product(numbers, repeat=NUMBER_OF_INTEGERS))
print(all_permutations)

helper_AND_variables = model.addVars(NUMBER_OF_HELPER_VARIABLES, vtype=g.GRB.BINARY, name="b1")
helper_OR_variables = model.addVars(NUMBER_OF_HELPER_VARIABLES, vtype=g.GRB.BINARY, name="b2")
helper_NAND_variables = model.addVars(NUMBER_OF_HELPER_VARIABLES, vtype=g.GRB.BINARY, name="b3")
helper_NOR_variables = model.addVars(NUMBER_OF_HELPER_VARIABLES, vtype=g.GRB.BINARY, name="b4")

# AND constraints
for index, permutation in enumerate(all_permutations):
    model.addConstr(0*int_set[permutation[0]] + 0*int_set[permutation[1]] + int_set[permutation[2]] <= -5 + M * (1 - helper_AND_variables[index]) ,f"AND 1.{permutation}")
    model.addConstr(0*int_set[permutation[0]] + 1*int_set[permutation[1]] + int_set[permutation[2]] <= -5 + M * (1 - helper_AND_variables[index]) ,f"AND 2.{permutation}")
    model.addConstr(1*int_set[permutation[0]] + 0*int_set[permutation[1]] + int_set[permutation[2]] <= -5 + M * (1 - helper_AND_variables[index]) ,f"AND 3.{permutation}")
    model.addConstr(1*int_set[permutation[0]] + 1*int_set[permutation[1]] + int_set[permutation[2]] >= 5 - M * (1 - helper_AND_variables[index]) ,f"AND 4.{permutation}")


model.addConstr(g.quicksum(helper_AND_variables[i] for i in range(NUMBER_OF_HELPER_VARIABLES)) >= 1, "AtLeastOneANDGroup")

# # OR constraints
for index, permutation in enumerate(all_permutations):
    model.addConstr(0*int_set[permutation[0]] + 0*int_set[permutation[1]] + int_set[permutation[2]] <= -5 + M * (1 - helper_OR_variables[index]) ,f"OR 1.{permutation}")
    model.addConstr(0*int_set[permutation[0]] + 1*int_set[permutation[1]] + int_set[permutation[2]] >= 5 - M * (1 - helper_OR_variables[index]) ,f"OR 2.{permutation}")
    model.addConstr(1*int_set[permutation[0]] + 0*int_set[permutation[1]] + int_set[permutation[2]] >= 5 - M * (1 - helper_OR_variables[index]) ,f"OR 3.{permutation}")
    model.addConstr(1*int_set[permutation[0]] + 1*int_set[permutation[1]] + int_set[permutation[2]] >= 5 - M * (1 - helper_OR_variables[index]) ,f"OR 4.{permutation}")


model.addConstr(g.quicksum(helper_OR_variables[i] for i in range(NUMBER_OF_HELPER_VARIABLES)) >= 1, "AtLeastOneORGroup")

# NAND constraints
for index, permutation in enumerate(all_permutations):
    model.addConstr(0*int_set[permutation[0]] + 0*int_set[permutation[1]] + int_set[permutation[2]] >= 5 - M * (1 - helper_NAND_variables[index]) ,f"NAND 1.{permutation}")
    model.addConstr(0*int_set[permutation[0]] + 1*int_set[permutation[1]] + int_set[permutation[2]] >= 5 - M * (1 - helper_NAND_variables[index]) ,f"NAND 2.{permutation}")
    model.addConstr(1*int_set[permutation[0]] + 0*int_set[permutation[1]] + int_set[permutation[2]] >= 5 - M * (1 - helper_NAND_variables[index]) ,f"NAND 3.{permutation}")
    model.addConstr(1*int_set[permutation[0]] + 1*int_set[permutation[1]] + int_set[permutation[2]] <= -5 + M * (1 - helper_NAND_variables[index]) ,f"NAND 4.{permutation}")

model.addConstr(g.quicksum(helper_NAND_variables[i] for i in range(NUMBER_OF_HELPER_VARIABLES)) >= 1, "AtLeastOneNANDGroup")

for index, permutation in enumerate(all_permutations):
    model.addConstr(0*int_set[permutation[0]] + 0*int_set[permutation[1]] + int_set[permutation[2]] >= 5 - M * (1 - helper_NOR_variables[index]) ,f"NOR 1.{permutation}")
    model.addConstr(0*int_set[permutation[0]] + 1*int_set[permutation[1]] + int_set[permutation[2]] <= -5 + M * (1 - helper_NOR_variables[index]) ,f"NOR 2.{permutation}")
    model.addConstr(1*int_set[permutation[0]] + 0*int_set[permutation[1]] + int_set[permutation[2]] <= -5 + M * (1 - helper_NOR_variables[index]) ,f"NOR 3.{permutation}")
    model.addConstr(1*int_set[permutation[0]] + 1*int_set[permutation[1]] + int_set[permutation[2]] <= -5 + M * (1 - helper_NOR_variables[index]) ,f"NOR 4.{permutation}")

model.addConstr(g.quicksum(helper_NOR_variables[i] for i in range(NUMBER_OF_HELPER_VARIABLES)) >= 1, "AtLeastOneNANDGroup")

# Optimize
model.optimize()


'''
The viable integers for representing the set of logic gates
'''
for index in range(NUMBER_OF_INTEGERS):
    print(f"The integer {index} is {int_set[index].x}")

'''
the index of where helper variable is 1 denotes represents the specific permutation, 
which is viable for representing representing the logic gate
'''
for index in range(NUMBER_OF_HELPER_VARIABLES):
    if (helper_AND_variables[index].x > 0.1):
        print(f"The permutation for AND is {all_permutations[index]}")
    if (helper_OR_variables[index].x > 0.1):
        print(f"The permutation for OR is {all_permutations[index]}")
    if (helper_NAND_variables[index].x > 0.1):
        print(f"The permutation for NAND is {all_permutations[index]}")
    if (helper_NOR_variables[index].x > 0.1):
        print(f"The permutation for NAND is {all_permutations[index]}")


[(0, 0, 0), (0, 0, 1), (0, 0, 2), (0, 1, 0), (0, 1, 1), (0, 1, 2), (0, 2, 0), (0, 2, 1), (0, 2, 2), (1, 0, 0), (1, 0, 1), (1, 0, 2), (1, 1, 0), (1, 1, 1), (1, 1, 2), (1, 2, 0), (1, 2, 1), (1, 2, 2), (2, 0, 0), (2, 0, 1), (2, 0, 2), (2, 1, 0), (2, 1, 1), (2, 1, 2), (2, 2, 0), (2, 2, 1), (2, 2, 2)]
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i7-10875H CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 109 rows, 30 columns and 309 nonzeros
Model fingerprint: 0x48156198
Variable types: 0 continuous, 30 integer (27 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+01]
  RHS range        [1e+00, 1e+02]
Presolve removed 66 rows and 15 columns
Presolve time: 0.00s
Presolved: 43 rows, 15 columns, 132 nonzeros
Variable types: 0 continuous, 15 integer (12 

In [20]:
import gurobipy as g
import itertools

def optimize_logic_gates(gates_to_include):
    NUMBER_OF_INTEGERS = 3
    NUMBER_OF_HELPER_VARIABLES = pow(NUMBER_OF_INTEGERS, NUMBER_OF_INTEGERS)
    M = 100  # Big-M

    model = g.Model()

    # Define integer set variables
    int_set = model.addVars(NUMBER_OF_INTEGERS, vtype=g.GRB.INTEGER, lb=-15, ub=15, name="x")

    # Generate all possible permutations with repetition of integer variables
    numbers = list(range(NUMBER_OF_INTEGERS))
    all_permutations = list(itertools.product(numbers, repeat=NUMBER_OF_INTEGERS))

    helper_variables = {}
    for gate in ['AND', 'OR', 'NAND', 'NOR', 'IMPLICATION', 'COVERSE_IMPLICATION', 'EQUIVALENCE', 'x_AND_NOT_y', 'NOT_x_AND_y']:
        if gate in gates_to_include:
            helper_variables[gate] = model.addVars(NUMBER_OF_HELPER_VARIABLES, vtype=g.GRB.BINARY, name=f"b_{gate}")

    # Define constraints based on gates to include
    for index, permutation in enumerate(all_permutations):
        if 'AND' in gates_to_include:
            add_and_constraints(model, int_set, permutation, index, helper_variables['AND'], M)
        if 'OR' in gates_to_include:
            add_or_constraints(model, int_set, permutation, index, helper_variables['OR'], M)
        if 'NAND' in gates_to_include:
            add_nand_constraints(model, int_set, permutation, index, helper_variables['NAND'], M)
        if 'NOR' in gates_to_include:
            add_nor_constraints(model, int_set, permutation, index, helper_variables['NOR'], M)
        if 'IMPLICATION' in gates_to_include:
            add_implication_constraints(model, int_set, permutation, index, helper_variables['IMPLICATION'], M)
        if 'COVERSE_IMPLICATION' in gates_to_include:
            add_converse_implication_constraints(model, int_set, permutation, index, helper_variables['COVERSE_IMPLICATION'], M)
        if 'EQUIVALENCE' in gates_to_include:
            add_equivalence_constraints(model, int_set, permutation, index, helper_variables['EQUIVALENCE'], M)
        if 'x_AND_NOT_y' in gates_to_include:
            add_x_AND_NOT_y_constraints(model, int_set, permutation, index, helper_variables['x_AND_NOT_y'], M)
        if 'NOT_x_AND_y' in gates_to_include:
            add_NOT_x_AND_y_constraints(model, int_set, permutation, index, helper_variables['NOT_x_AND_y'], M)

    # Ensure at least one permutation for each gate is valid
    for gate in gates_to_include:
        model.addConstr(g.quicksum(helper_variables[gate][i] for i in range(NUMBER_OF_HELPER_VARIABLES)) >= 1, f"AtLeastOne{gate}Group")

    # Optimize
    model.optimize()

    # Print results
    for index in range(NUMBER_OF_INTEGERS):
        print(f"The integer {index} is {int_set[index].x}")

    for gate in gates_to_include:
        for index in range(NUMBER_OF_HELPER_VARIABLES):
            if helper_variables[gate][index].x > 0.1:
                print(f"The permutation for {gate} is {all_permutations[index]}")

def add_and_constraints(model, int_set, permutation, index, helper_variable, M):
    model.addConstr(0*int_set[permutation[0]] + 0*int_set[permutation[1]] + int_set[permutation[2]] <= -5 + M * (1 - helper_variable[index]), f"AND 1.{permutation}")
    model.addConstr(0*int_set[permutation[0]] + 1*int_set[permutation[1]] + int_set[permutation[2]] <= -5 + M * (1 - helper_variable[index]), f"AND 2.{permutation}")
    model.addConstr(1*int_set[permutation[0]] + 0*int_set[permutation[1]] + int_set[permutation[2]] <= -5 + M * (1 - helper_variable[index]), f"AND 3.{permutation}")
    model.addConstr(1*int_set[permutation[0]] + 1*int_set[permutation[1]] + int_set[permutation[2]] >= 5 - M * (1 - helper_variable[index]), f"AND 4.{permutation}")

def add_or_constraints(model, int_set, permutation, index, helper_variable, M):
    model.addConstr(0*int_set[permutation[0]] + 0*int_set[permutation[1]] + int_set[permutation[2]] <= -5 + M * (1 - helper_variable[index]) ,f"OR 1.{permutation}")
    model.addConstr(0*int_set[permutation[0]] + 1*int_set[permutation[1]] + int_set[permutation[2]] >= 5 - M * (1 - helper_variable[index]) ,f"OR 2.{permutation}")
    model.addConstr(1*int_set[permutation[0]] + 0*int_set[permutation[1]] + int_set[permutation[2]] >= 5 - M * (1 - helper_variable[index]) ,f"OR 3.{permutation}")
    model.addConstr(1*int_set[permutation[0]] + 1*int_set[permutation[1]] + int_set[permutation[2]] >= 5 - M * (1 - helper_variable[index]) ,f"OR 4.{permutation}")

def add_nand_constraints(model, int_set, permutation, index, helper_variable, M):
    model.addConstr(0*int_set[permutation[0]] + 0*int_set[permutation[1]] + int_set[permutation[2]] >= 5 - M * (1 - helper_variable[index]) ,f"NAND 1.{permutation}")
    model.addConstr(0*int_set[permutation[0]] + 1*int_set[permutation[1]] + int_set[permutation[2]] >= 5 - M * (1 - helper_variable[index]) ,f"NAND 2.{permutation}")
    model.addConstr(1*int_set[permutation[0]] + 0*int_set[permutation[1]] + int_set[permutation[2]] >= 5 - M * (1 - helper_variable[index]) ,f"NAND 3.{permutation}")
    model.addConstr(1*int_set[permutation[0]] + 1*int_set[permutation[1]] + int_set[permutation[2]] <= -5 + M * (1 - helper_variable[index]) ,f"NAND 4.{permutation}")

def add_nor_constraints(model, int_set, permutation, index, helper_variable, M):
    model.addConstr(0*int_set[permutation[0]] + 0*int_set[permutation[1]] + int_set[permutation[2]] >= 5 - M * (1 - helper_variable[index]) ,f"NOR 1.{permutation}")
    model.addConstr(0*int_set[permutation[0]] + 1*int_set[permutation[1]] + int_set[permutation[2]] <= -5 + M * (1 - helper_variable[index]) ,f"NOR 2.{permutation}")
    model.addConstr(1*int_set[permutation[0]] + 0*int_set[permutation[1]] + int_set[permutation[2]] <= -5 + M * (1 - helper_variable[index]) ,f"NOR 3.{permutation}")
    model.addConstr(1*int_set[permutation[0]] + 1*int_set[permutation[1]] + int_set[permutation[2]] <= -5 + M * (1 - helper_variable[index]) ,f"NOR 4.{permutation}")

def add_implication_constraints(model, int_set, permutation, index, helper_variable, M):
    model.addConstr(0*int_set[permutation[0]] + 0*int_set[permutation[1]] + int_set[permutation[2]] >= 5 - M * (1 - helper_variable[index]) ,f"IMPLICATION 1.{permutation}")
    model.addConstr(0*int_set[permutation[0]] + 1*int_set[permutation[1]] + int_set[permutation[2]] >= 5 - M * (1 - helper_variable[index]) ,f"IMPLICATION 2.{permutation}")
    model.addConstr(1*int_set[permutation[0]] + 0*int_set[permutation[1]] + int_set[permutation[2]] <= -5 + M * (1 - helper_variable[index]) ,f"IMPLICATION 3.{permutation}")
    model.addConstr(1*int_set[permutation[0]] + 1*int_set[permutation[1]] + int_set[permutation[2]] >= 5 - M * (1 - helper_variable[index]) ,f"IMPLICATION 4.{permutation}")

def add_converse_implication_constraints(model, int_set, permutation, index, helper_variable, M):
    model.addConstr(0*int_set[permutation[0]] + 0*int_set[permutation[1]] + int_set[permutation[2]] >= 5 - M * (1 - helper_variable[index]) ,f"COVERSE_IMPLICATION 1.{permutation}")
    model.addConstr(0*int_set[permutation[0]] + 1*int_set[permutation[1]] + int_set[permutation[2]] <= -5 + M * (1 - helper_variable[index]) ,f"COVERSE_IMPLICATION 2.{permutation}")
    model.addConstr(1*int_set[permutation[0]] + 0*int_set[permutation[1]] + int_set[permutation[2]] >= 5 - M * (1 - helper_variable[index]) ,f"COVERSE_IMPLICATION 3.{permutation}")
    model.addConstr(1*int_set[permutation[0]] + 1*int_set[permutation[1]] + int_set[permutation[2]] >= 5 - M * (1 - helper_variable[index]) ,f"COVERSE_IMPLICATION 4.{permutation}")

def add_equivalence_constraints(model, int_set, permutation, index, helper_variable, M):
    model.addConstr(0*int_set[permutation[0]] + 0*int_set[permutation[1]] + int_set[permutation[2]] >= 5 - M * (1 - helper_variable[index]) ,f"EQUIVALENCE 1.{permutation}")
    model.addConstr(0*int_set[permutation[0]] + 1*int_set[permutation[1]] + int_set[permutation[2]] <= -5 + M * (1 - helper_variable[index]) ,f"EQUIVALENCE 2.{permutation}")
    model.addConstr(1*int_set[permutation[0]] + 0*int_set[permutation[1]] + int_set[permutation[2]] <= -5 + M * (1 - helper_variable[index]) ,f"EQUIVALENCE 3.{permutation}")
    model.addConstr(1*int_set[permutation[0]] + 1*int_set[permutation[1]] + int_set[permutation[2]] >= 5 - M * (1 - helper_variable[index]) ,f"EQUIVALENCE 4.{permutation}")

def add_x_AND_NOT_y_constraints(model, int_set, permutation, index, helper_variable, M):
    model.addConstr(0*int_set[permutation[0]] + 0*int_set[permutation[1]] + int_set[permutation[2]] <= -5 + M * (1 - helper_variable[index]) ,f"x_AND_NOT_y 1.{permutation}")
    model.addConstr(0*int_set[permutation[0]] + 1*int_set[permutation[1]] + int_set[permutation[2]] <= -5 + M * (1 - helper_variable[index]) ,f"x_AND_NOT_y 2.{permutation}")
    model.addConstr(1*int_set[permutation[0]] + 0*int_set[permutation[1]] + int_set[permutation[2]] >= 5 - M * (1 - helper_variable[index]) ,f"x_AND_NOT_y 3.{permutation}")
    model.addConstr(1*int_set[permutation[0]] + 1*int_set[permutation[1]] + int_set[permutation[2]] <= -5 + M * (1 - helper_variable[index]) ,f"x_AND_NOT_y 4.{permutation}")

def add_NOT_x_AND_y_constraints(model, int_set, permutation, index, helper_variable, M):
    model.addConstr(0*int_set[permutation[0]] + 0*int_set[permutation[1]] + int_set[permutation[2]] <= -5 + M * (1 - helper_variable[index]) ,f"NOT_x_AND_y 1.{permutation}")
    model.addConstr(0*int_set[permutation[0]] + 1*int_set[permutation[1]] + int_set[permutation[2]] >= 5 - M * (1 - helper_variable[index]) ,f"NOT_x_AND_y 2.{permutation}")
    model.addConstr(1*int_set[permutation[0]] + 0*int_set[permutation[1]] + int_set[permutation[2]] <= -5 + M * (1 - helper_variable[index]) ,f"NOT_x_AND_y 3.{permutation}")
    model.addConstr(1*int_set[permutation[0]] + 1*int_set[permutation[1]] + int_set[permutation[2]] <= -5 + M * (1 - helper_variable[index]) ,f"NOT_x_AND_y 4.{permutation}")

# Example usage
optimize_logic_gates({'NOT_x_AND_y'})


Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i7-10875H CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 109 rows, 30 columns and 309 nonzeros
Model fingerprint: 0xd60b3303
Variable types: 0 continuous, 30 integer (27 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 1e+02]
Presolve removed 72 rows and 15 columns
Presolve time: 0.00s
Presolved: 37 rows, 15 columns, 114 nonzeros
Variable types: 0 continuous, 15 integer (12 binary)
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 16 (of 16 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.00000

## Possible activation functions
- sigmoid
- threshold function
- hyperbolic tanget